In [182]:
import requests
from bs4 import BeautifulSoup

Import web page data.

In [183]:
URL = 'http://pibs.nats.co.uk/operational/pibs/pib4.shtml'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
# Filter the other HTML elements.
notavs_dirty = soup.find_all(class_="notam")
# Parse to return only the notav elements.
notavs = [notav.table.tr.find("td", class_='') for notav in notavs_dirty]
print(notavs)

[<td>
<div>
<b>Q) </b>EGXX/QSEXX/IV/B/E/000/100/5502N00141W040</div>
<pre>ENR 1.6 ATS UNITS PARTICIPATING IN LOWER AIRSPACE RADAR - CHANGE 
NEWCASTLE LOWER AIRSPACE RADAR SERVICE RADIUS FROM 60NM TO 40NM.
UK AIP ENR 1.6, ENR 6-11 REFERS.</pre>
<div class="notamid">B2511/21</div>
<div>
<span class="endgap">
<b>FROM: </b>15 OCT 2021 15:00</span>
<span>
<b>TO: </b>PERM</span>
</div>
</td>, <td>
<div>
<b>Q) </b>EGPX/QOBCE/IV/M/E/000/008/5724N00624W001</div>
<pre>MAST AT PSN 572358N 0062352W (VCY BALMEANACH, ISLE OF SKYE). UP
TO 328FT AGL/772FT AMSL. FOR INFO CONTACT 01501 785088. ON EXPIRY 
OF THIS NOTAM DETAILS WILL BE INCLUDED IN THE UK AIP ENR 5.4. 
2021-11-0487/AS8</pre>
<div class="notamid">N0205/21</div>
<div>
<span class="endgap">
<b>FROM: </b>23 NOV 2021 00:01</span>
<span>
<b>TO: </b>21 FEB 2022 23:59</span>
</div>
</td>, <td>
<div>
<b>Q) </b>EGPX/QOBCE/IV/M/AE/000/003/5558N00324W001</div>
<div>
<b>A) </b>EGPH </div>
<pre>CRANE OPR 555730N 0032338W (KIRKLISTON), MAX HGT 282FT AMSL

Extract NOTAVs and locations. 

In [184]:
notav_dict = {}
for notav in notavs:
    # Get NOTAV ID
    notav_id = notav.find("div", class_='notamid').text
    # Get the latitude and longitude block.
    notav_latlong_text = notav.div.text.split('/')[-1]
    # Get latitude.
    notav_lat = notav_latlong_text[0 : 4]
    # Get longitude.
    notav_long = notav_latlong_text[6 : 10]
    # Get text description.
    notav_text = notav.pre.text
    # Start and end time.
    notav_start_time = notav.find("span", class_="endgap").text.split('FROM: ')[1]
    notav_end_time = notav.find_all("span")[1].text.split('TO: ')[1]
    # Append to dictionary.
    notav_dict[notav_id] = {
            'latitude': notav_lat,
            'longitude': notav_long,
            'description': notav_text,
            'start_time': notav_start_time,
            'end_time': notav_end_time
    }
    
print(list(notav_dict.items())[0 : 5])

[('B2511/21', {'latitude': '5502', 'longitude': '0141', 'description': 'ENR 1.6 ATS UNITS PARTICIPATING IN LOWER AIRSPACE RADAR - CHANGE \nNEWCASTLE LOWER AIRSPACE RADAR SERVICE RADIUS FROM 60NM TO 40NM.\nUK AIP ENR 1.6, ENR 6-11 REFERS.', 'start_time': '15 OCT 2021 15:00', 'end_time': 'PERM'}), ('N0205/21', {'latitude': '5724', 'longitude': '0624', 'description': 'MAST AT PSN 572358N 0062352W (VCY BALMEANACH, ISLE OF SKYE). UP\nTO 328FT AGL/772FT AMSL. FOR INFO CONTACT 01501 785088. ON EXPIRY \nOF THIS NOTAM DETAILS WILL BE INCLUDED IN THE UK AIP ENR 5.4. \n2021-11-0487/AS8', 'start_time': '23 NOV 2021 00:01', 'end_time': '21 FEB 2022 23:59'}), ('A4155/21', {'latitude': '5558', 'longitude': '0324', 'description': 'CRANE OPR 555730N 0032338W (KIRKLISTON), MAX HGT 282FT AMSL, \n131FT AGL. CRANE WILL BE LOWERED DURING LVP.', 'start_time': '03 DEC 2021 08:00', 'end_time': '02 MAR 2022 17:00'}), ('N0220/21', {'latitude': '5637', 'longitude': '0142', 'description': 'OBST WINDFARM WI 4NM RAD

Plot NOTAVs on a map.

In [192]:
import pandas
import nbformat
import plotly.express as px
df = px.data.gapminder().query("year==2007")
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     projection="natural earth")
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed